In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
import geopandas as gpd
from trajectories_clustering import TrajectoryClustering
from trajectory_visualizer_matplot import TrajectoryVisualizerMatplot

In [3]:
df = pd.read_csv('./data/input/gps_logs.csv')
df['trip'].value_counts()

1243    2695
1809    2411
1558    2125
1167    2053
1319    1959
        ... 
977      364
1505     359
1428     358
722      355
467      230
Name: trip, Length: 169, dtype: int64

In [4]:
trajetorias = []
trips_ids = []
for trip_id, frame in df.groupby('trip'):
    rota = list(zip(frame['lat'], frame['lon']))
    trajetorias.append(rota)
    trips_ids.append(trip_id)

In [5]:
import folium

print(trips_ids[1])
# cluster A = 0, 1, 2, 4, 7, 8
# cluster B = 5, 28, 30, 35, 40
# cluster C = 6, 9, 10, 11, 12
rota = trajetorias[40]
mapa = folium.Map(location=rota[0], zoom_start=11)

for ponto in rota:
    folium.Circle(ponto, fill=True, radius=4, color='blue', fill_color='blue').add_to(mapa)

mapa

54


In [6]:
# cluster A = 0, 1, 2, 4, 7, 8
# cluster B = 5, 28, 30, 35, 40
# cluster C = 6, 9, 10, 11, 12
trajetorias_s = [trajetorias[i] for i in [0,5,6,1,28,9,2,30,10,4,35,11,7,40,12,8]]
# 0,1,2,0,1,2
for t in trajetorias_s:
    print(len(t))

985
528
1118
832
541
638
954
619
787
1007
478
668
584
571
704
729


In [7]:
trajectories = [np.array(x) for x in trajetorias_s]

In [8]:
trajectories_xy = TrajectoryClustering.convert_lat_lon_to_xy(trajectories)

In [9]:
trajectories_reduced = [TrajectoryClustering.reduce_polyline_points_by_rdp(p) for p in trajectories_xy]

In [10]:
data = [[len(t), len(trajectories_reduced[i])] for i, t in enumerate(trajectories)]
red = pd.DataFrame(data=data, columns=["original data points", "after RDP"])
red.index.name = "traj #"
red.head(16)

,original data points,after RDP
traj #,,
0,985,24
1,528,21
2,1118,8
3,832,13
4,541,20
5,638,9
6,954,21
7,619,20
8,787,17


In [11]:
dist_mat_reduced = TrajectoryClustering.compute_distance_matrix(trajectories_reduced)

In [18]:
labels = TrajectoryClustering.clustering_by_dbscan(dist_mat_reduced, eps=1500)
labels
# Nesse caso, o valor de eps que funciona é entre 1500 e 3500

array([0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0], dtype=int64)